In [32]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import squarify
sns.set(style="whitegrid")
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import log_loss 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

In [33]:
test = pd.read_csv('test_listo.csv')
train = pd.read_csv('train_listo.csv')

In [34]:
train.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,precio_por_items
0,0,1,27761,Europa,None,1,1,1,1,0,...,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None,None,NaN,5272800.0,5963536.80,5963536.80
1,1,1,27760,Europa,None,0,0,0,0,0,...,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None,None,NaN,48230.0,54548.13,54548.13
2,2,1,27446,América,NW America,0,0,0,0,0,...,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otoño,83865.6,83865.60,83865.60
3,3,1,16808,América,NW America,1,0,1,0,0,...,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,7421881.5,7421881.50,7421881.50
4,4,1,16805,América,NW America,1,0,1,0,0,...,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,13357192.5,13357192.50,13357192.50


In [35]:
test.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,precio_por_items
0,10689,3,6140,Europa,Germany,1,1,1,1,0,...,EUR,162240.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.8890,138516.9630
1,10689,3,6146,Europa,Germany,1,1,1,1,0,...,EUR,78624.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.8890,138516.9630
2,10689,3,6151,Europa,Germany,1,1,1,1,0,...,EUR,126555.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.8890,138516.9630
3,10690,5,6118,Europa,Germany,1,1,1,1,0,...,EUR,243360.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,171410.6277
4,10690,5,6124,Europa,Germany,1,1,1,1,0,...,EUR,157248.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,171410.6277


In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16883 entries, 0 to 16882
Data columns (total 58 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Opportunity_ID                      16883 non-null  int64  
 1   Cantidad de Items Asociados         16883 non-null  int64  
 2   ID                                  16883 non-null  int64  
 3   Region                              16883 non-null  object 
 4   Territory                           16883 non-null  object 
 5   Pricing, Delivery_Terms_Quote_Appr  16883 non-null  int64  
 6   Pricing, Delivery_Terms_Approved    16883 non-null  int64  
 7   Bureaucratic_Code_0_Approval        16883 non-null  int64  
 8   Bureaucratic_Code_0_Approved        16883 non-null  int64  
 9   Submitted_for_Approval              16883 non-null  int64  
 10  Bureaucratic_Code                   16883 non-null  object 
 11  Account_Created_Date                16883

In [37]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 57 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Opportunity_ID                      2551 non-null   int64  
 1   Cantidad de Items Asociados         2551 non-null   int64  
 2   ID                                  2551 non-null   int64  
 3   Region                              2551 non-null   object 
 4   Territory                           2551 non-null   object 
 5   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64  
 6   Pricing, Delivery_Terms_Approved    2551 non-null   int64  
 7   Bureaucratic_Code_0_Approval        2551 non-null   int64  
 8   Bureaucratic_Code_0_Approved        2551 non-null   int64  
 9   Submitted_for_Approval              2551 non-null   int64  
 10  Bureaucratic_Code                   2551 non-null   object 
 11  Account_Created_Date                2551 no

In [38]:
len(set(train.Opportunity_Name))

9791

In [39]:
train.loc[train['Stage']=='Closed Won','Stage']=1
train.loc[train['Stage']=='Closed Lost','Stage']=0

In [40]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

In [41]:
def LabelEncord_categorical(df):
    categorical_features = ['Region','Territory','Bureaucratic_Code','Source ','Billing_Country', 'Account_Name','Account_Owner',\
                            'Opportunity_Owner','Account_Type', 'Opportunity_Type', 'Quote_Type','Delivery_Quarter', \
                            'Delivery_Terms','Last_Modified_By',\
                            'Product_Family', 'Product_Name','ASP_Currency','Prod_Category_A', 'Hemisferio', 'Estación']
    for features in categorical_features:
        le = LabelEncoder()
        df[features] = le.fit_transform(df[features].astype(str))
    return df

def dummies(df):
    categorical_features = ["year","provincia","tipodepropiedad",'ciudad']
    for features in categorical_features:
        dummies =  pd.get_dummies(df[features])
        df = pd.concat([df, dummies],axis=1)
    return df

def pre_processing(df):
    df = LabelEncord_categorical(df)
    return df

#df = pre_processing(df)
train = pre_processing(train)
test = pre_processing(test)

In [42]:
train.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,precio_por_items
0,0,1,27761,2,46,1,1,1,1,0,...,5272800.0,EUR,5272800.0,0,0,2,4,5272800.0,5963536.80,5963536.80
1,1,1,27760,2,46,0,0,0,0,0,...,48230.0,EUR,48230.0,1,0,2,4,48230.0,54548.13,54548.13
2,2,1,27446,0,43,0,0,0,0,0,...,83865.6,USD,83865.6,1,0,0,1,83865.6,83865.60,83865.60
3,3,1,16808,0,43,1,0,1,0,0,...,7421881.5,USD,7421881.5,0,0,0,1,7421881.5,7421881.50,7421881.50
4,4,1,16805,0,43,1,0,1,0,0,...,13357192.5,USD,13357192.5,0,0,0,1,13357192.5,13357192.50,13357192.50


In [43]:
test.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,precio_por_items
0,10689,3,6140,2,15,1,1,1,1,0,...,EUR,162240.0,EUR,367419.0,0,2,2,367419.0,415550.8890,138516.9630
1,10689,3,6146,2,15,1,1,1,1,0,...,EUR,78624.0,EUR,367419.0,0,2,2,367419.0,415550.8890,138516.9630
2,10689,3,6151,2,15,1,1,1,1,0,...,EUR,126555.0,EUR,367419.0,0,2,2,367419.0,415550.8890,138516.9630
3,10690,5,6118,2,15,1,1,1,1,0,...,EUR,243360.0,EUR,757783.5,0,2,2,757783.5,857053.1385,171410.6277
4,10690,5,6124,2,15,1,1,1,1,0,...,EUR,157248.0,EUR,757783.5,0,2,2,757783.5,857053.1385,171410.6277


In [44]:
#train.fillna(0,inplace=True)
#test.fillna(0,inplace=True)
#train.rename(columns={'Estación':'Estacion'},inplace=True)
#test.rename(columns={'Estación':'Estacion'},inplace=True)

In [45]:
categorical_features = ['Region','Territory','Bureaucratic_Code','Source ','Billing_Country', 'Account_Name','Account_Owner',\
                            'Opportunity_Owner','Account_Type', 'Opportunity_Type', 'Quote_Type','Delivery_Quarter', \
                            'Delivery_Terms','Last_Modified_By',\
                            'Product_Family', 'Product_Name','ASP_Currency','Prod_Category_A', 'Hemisferio', 'Estacion']

In [46]:
# Aca saco las columnas(labels que no quiero que las tome mi algoritmo)
features = list(set(train.columns) - set(['ID','Stage','Sales_Contract_No','Opportunity_ID','Opportunity_Created_Date','Account_Created_Date','Opportunity_Name',\
                                        'Price', 'Currency', 'ASP', 'ASP_(converted)_Currency', 'ASP_(converted)',\
                                        'Planned_Delivery_Start_Date','Brand','Size', 'Planned_Delivery_End_Date', 'Month',\
                                         'Delivery_Year', 'Actual_Delivery_Date','Product_Type','Product_Category_B',\
                                        'Total_Amount_Currency', 'Total_Amount','Last_Activity',\
                                        'Quote_Expiry_Date', 'Last_Modified_Date',\
                                        'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount','Account_Created_Date',]))
features

['Opportunity_Type',
 'Territory',
 'Estación',
 'Product_Family',
 'total_en_dolares',
 'Opportunity_Owner',
 'Pricing, Delivery_Terms_Approved',
 'Source ',
 'precio_por_items',
 'Delivery_Quarter',
 'Delivery_Terms',
 'ASP_Currency',
 'Product_Name',
 'Account_Type',
 'Cantidad de Items Asociados',
 'Region',
 'Prod_Category_A',
 'Bureaucratic_Code',
 'TRF',
 'Hemisferio',
 'Account_Owner',
 'Submitted_for_Approval',
 'Pricing, Delivery_Terms_Quote_Appr',
 'Billing_Country',
 'Bureaucratic_Code_0_Approved',
 'Bureaucratic_Code_0_Approval',
 'Account_Name',
 'Total_moneda_local',
 'Quote_Type',
 'Last_Modified_By']

In [47]:
train = train.drop_duplicates(subset='Opportunity_ID',keep='first') 
test = test.drop_duplicates(subset='Opportunity_ID',keep='first') 

In [48]:
X = train[features]
Y = train['Stage']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [49]:
x_train

,Opportunity_Type,Territory,Estación,Product_Family,total_en_dolares,Opportunity_Owner,"Pricing, Delivery_Terms_Approved",Source,precio_por_items,Delivery_Quarter,...,Account_Owner,Submitted_for_Approval,"Pricing, Delivery_Terms_Quote_Appr",Billing_Country,Bureaucratic_Code_0_Approved,Bureaucratic_Code_0_Approval,Account_Name,Total_moneda_local,Quote_Type,Last_Modified_By
4144,1,43,3,89,1.028517e+06,52,1,2,1.028517e+06,3,...,28,0,1,77,1,1,59,1028517.00,1,31
2311,11,54,0,55,2.324851e+06,19,1,12,2.324851e+06,2,...,20,0,1,77,1,1,1381,2324851.20,1,31
13949,1,24,2,9,2.340852e+06,47,1,12,3.901420e+05,2,...,41,0,1,27,0,0,551,2069718.75,1,26
9495,1,31,3,35,1.350240e+05,51,1,0,1.350240e+05,0,...,45,0,1,36,1,1,1595,135024.00,1,31
12500,11,24,0,9,2.712138e+06,26,0,12,4.520230e+05,1,...,22,0,1,27,0,1,853,2398000.00,1,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,4,3,3,3,6.686820e+04,8,1,13,6.686820e+04,0,...,25,0,1,3,1,1,555,94500.00,1,31
7423,22,36,3,36,1.033169e+05,7,0,0,1.033169e+05,3,...,6,0,0,45,0,0,990,91350.00,1,31
5563,13,31,1,16,1.120000e+02,37,0,0,1.120000e+02,1,...,32,0,0,36,0,0,1192,112.00,1,35
12861,11,24,0,36,6.366784e+06,26,1,0,2.122261e+06,3,...,22,0,1,27,1,1,1301,5629340.00,1,24


In [55]:
xgb1 = xgb.XGBClassifier(n_jobs=-1,learning_rate=0.03,n_estimators=150)

In [56]:
xgb1.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=150,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [21]:
preds=xgb1.predict_proba(test[features])[:,1]

In [22]:
len(preds)

1567

In [23]:
len(test)

1567

In [26]:
test['target']=preds

In [27]:
test['target'].value_counts()

0.788807    23
0.718909    12
0.819603    12
0.746529    11
0.480853    10
            ..
0.339487     1
0.698504     1
0.301398     1
0.559825     1
0.692974     1
Name: target, Length: 1096, dtype: int64

In [28]:
test[['Opportunity_ID', 'target']]

,Opportunity_ID,target
0,10689,0.481021
3,10690,0.449850
8,10691,0.653310
9,10692,0.307048
15,10693,0.774275
...,...,...
2545,12364,0.727120
2547,12365,0.742492
2548,12366,0.662245
2549,12367,0.406754


In [29]:
test[['Opportunity_ID', 'target']].to_csv("subidas/xgb_label_enc_v1.csv", index=False)